# Single Subject Segmentation Q1K

In [ ]:
##use these commented parameters for testing this notebook outside of the automated loop of q1k_automated_reports.ipynb
#subject_id = "10062P"
#task_id = "VEP"
#session_id = "01"
#run_id = "1"
#project_path = "/project/def-emayada/q1k/experimental/HSJ/"
#pylossless_path = "derivatives/pylossless/"
#postproc_path = "derivatives/postproc/"
#et_sync = True

#use these parameters when executing this notebook from the automation notebook..
subject_id = ""
task_id = ""
session_id = ""
run_id = ""
project_path = "/project/def-emayada/q1k/experimental/"
pylossless_path = "derivatives/pylossless/"
sync_loss_path = "derivatives/sync_loss/"


In [ ]:
# import packages
import mne
import mne_bids
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt

from pathlib import Path
import shutil
import pylossless as ll
import q1k_sync_loss_tools as qslt
import warnings
warnings.filterwarnings('ignore')


from matplotlib import pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
import plotly.io as pio
pio.renderers.default = "plotly_mimetype+notebook"


In [ ]:
if task_id in {'VEP', 'GO', 'NSP', 'PLR', 'VS'}:
    et_sync = True
    print('et_sync enabled...')
else:
    et_sync = False
    print('et_sync disabled...')

# Set the parameters and read the raw and pylossless data.

In [ ]:
# create the BIDS path object..
bids_path = mne_bids.BIDSPath(
    subject=subject_id, session=session_id, task=task_id, run="1", datatype="eeg", suffix="eeg",root=project_path
)
print(bids_path)# Read the BIDS pylossless output file..
eeg_raw = mne_bids.read_raw_bids(bids_path=bids_path, verbose=False)
device_info=eeg_raw.info['device_info']

# create the BIDS path object..
bids_ll_path = mne_bids.BIDSPath(
    subject=subject_id, session=session_id, task=task_id, run="1", datatype="eeg", suffix="eeg",root=project_path + pylossless_path
)
#print(bids_ll_path)# Read the BIDS pylossless output file..
#eeg_ll_raw = mne_bids.read_raw_bids(bids_path=bids_ll_path, verbose=False)

ll_state = ll.LosslessPipeline()
ll_state = ll_state.load_ll_derivative(bids_ll_path)
eeg_ll_raw = ll_state.raw.copy()

In [ ]:
#attempts to overwrite data array with original unfiltered signals... 

#insert the original data signals into the lossles state...
# Get the time range of raw_crop
start_time = eeg_ll_raw.times[0]  # Start time in seconds
end_time = eeg_ll_raw.times[-1]  # End time in seconds

# Crop eeg_raw to the time range of eeg_ll_raw
eeg_raw = eeg_raw.copy().crop(tmin=start_time, tmax=end_time)

# Overwrite annotations
eeg_raw.set_annotations(eeg_ll_raw.annotations)

In [ ]:
print(np.unique(eeg_raw.annotations.description))

In [ ]:
eeg_raw.plot(start=0, duration=20, scalings=dict(eeg=1e-3))

In [ ]:
eeg_raw.load_data()
eog_chans = ['E125', 'E126', 'E127', 'E128']
eeg_raw.info['bads'].extend(eog_chans)
eeg_raw = eeg_raw.filter(l_freq=1.0, h_freq=90.0, picks='eeg')
eeg_raw.notch_filter(freqs=60, picks='eeg', method='fir', fir_design='firwin')

In [ ]:
eeg_raw.plot(start=0, duration=20, scalings=dict(eeg=1e-4))

# Interpret the annotations, apply the data selections and insert the ET signals.

In [ ]:
# Get the events form the annotations
eeg_events, eeg_event_dict  = mne.events_from_annotations(eeg_raw)

In [ ]:
# peak... at the EEG event scatter plot.. event time stamp by label index
fig=px.scatter(x=eeg_events[:,0],y=eeg_events[:,2])
fig.update_layout(title='Original EEG event times')
fig.update_xaxes(title_text='Time of event(ms)')
fig.update_yaxes(title_text='Event index')
py.iplot(fig)
fig.write_html("eeg_event_times.html")

In [ ]:
bids_ll_path_str = str(bids_ll_path.fpath)
bids_ll_path_str

In [ ]:
if et_sync:
    #read the raw ET fif file from the BIDS root directory
    bids_ll_path_str=str(bids_ll_path.fpath)
    et_bids_path = bids_ll_path_str.replace(".edf", ".fif")
    et_bids_path = et_bids_path.replace("eeg", "et")
    et_bids_path = et_bids_path.replace("derivatives/pylossless/", "")
    print("et bids path: " + et_bids_path)
    
    et_raw = mne.io.read_raw_fif(et_bids_path, preload=True)
    et_sfreq = et_raw.info['sfreq'] 

    et_events, et_event_dict  = mne.events_from_annotations(et_raw)
    
    #interpolate the eye channels during blinks..
    mne.preprocessing.eyetracking.interpolate_blinks(et_raw, match=('BAD_blink','BAD_ACQ_SKIP'),buffer=(0.05, 0.2), interpolate_gaze=True)
    
    #replace Nans with 0s..
    data = et_raw.get_data()
    data[np.isnan(data)] = 0
    et_raw._data = data

In [ ]:
if et_sync:
    et_raw.plot(start=0, duration=20, scalings=dict(eyegaze=1e2,pupil=1e3))

In [ ]:
if et_sync:
    #get the sync_time event indexes..
    eeg_sync_time_value = eeg_event_dict['eeg_sync_time']
    et_sync_time_value = et_event_dict['et_sync_time']
    # Filter rows where the third column matches the 'sync_time' value
    eeg_syncs = eeg_events[eeg_events[:, 2] == eeg_sync_time_value]
    et_syncs = et_events[et_events[:, 2] == et_sync_time_value]
    #convert the event sample index to time (seconds)...
    eeg_sfreq = eeg_raw.info['sfreq']
    eeg_sync_times = eeg_syncs[:, 0] / eeg_sfreq
    et_sfreq = et_raw.info['sfreq']
    et_sync_times = et_syncs[:, 0] / et_sfreq

In [ ]:
eeg_raw.info

In [ ]:
if et_sync:
    et_raw.info

In [ ]:
if et_sync:
    #combine the EEG and ET data..
    eeg_raw, et_raw = qslt.eeg_et_combine(eeg_raw, et_raw, eeg_sync_times, et_sync_times, eeg_events, eeg_event_dict, et_events, et_event_dict)

In [ ]:
np.unique(eeg_raw.annotations.description)

In [ ]:
if et_sync:
    # Define a few channel groups of interest and plot the data\n",
    frontal = ["E11"]
    occipital = ["E62"]
    din = ["DIN"]
    pupil = ["pupil_left"]
    #x_pos = ["xpos_left"]
    #y_pos = ["ypos_left"]

    scale_dict = dict(eeg=1e-4, misc=1e3)

    # picks must be numeric (not string) when passed to `raw.plot(..., order=)
    picks_idx = mne.pick_channels(eeg_raw.ch_names, din + frontal + occipital + pupil, ordered=True)
    eeg_raw.plot(start=0,duration=20,order=picks_idx, scalings=scale_dict)

In [ ]:
if et_sync:
    # peak... at the ET event scatter plot.. event time stamp by label index
    fig=px.scatter(x=et_events[:,0],y=et_events[:,2])
    fig.update_layout(title='Original ET event times')
    fig.update_xaxes(title_text='Time of event(ms)')
    fig.update_yaxes(title_text='Event index')
    py.iplot(fig)
    fig.write_html("et_event_times.html")

In [ ]:
#apply the lossless annotations to the synced EEG and ET data..
eeg_loss_raw = qslt.apply_ll(bids_ll_path, ll_state, eeg_raw)
eeg_loss_raw

In [ ]:
if et_sync:
    # Define a few channel groups of interest and plot the data\n",
    frontal = ["E11"]
    occipital = ["E62"]
    din = ["DIN"]
    pupil = ["pupil_left"]
    #x_pos = ["xpos_left"]
    #y_pos = ["ypos_left"]

    #scale_dict = dict(eeg=1e-4, misc=1e3)
    scale_dict = dict(eeg=1e-3, misc=1e3)

    # picks must be numeric (not string) when passed to `raw.plot(..., order=)
    picks_idx = mne.pick_channels(eeg_loss_raw.ch_names, din + frontal + occipital + pupil, ordered=True)
    eeg_loss_raw.plot(start=0,duration=0,order=picks_idx, scalings=scale_dict)

In [ ]:
if et_sync:
    #rename the eyegaze and pupil channel types to misc for mne-bids compatibility..
    mapping = {
        ch_name: 'misc'
        for ch_name, ch_type in zip(eeg_raw.info['ch_names'], eeg_raw.get_channel_types())
        if ch_type in ['eyegaze', 'pupil']
    }

    # Rename the channel types
    eeg_raw.set_channel_types(mapping)


In [ ]:
#show channel types...
channel_types = eeg_loss_raw.get_channel_types()
print("Channel Types:", channel_types)
print("Channel Names:", eeg_loss_raw.info['ch_names'])

In [ ]:
np.unique(eeg_loss_raw.annotations.description)

In [ ]:
eeg_loss_events, eeg_loss_event_dict  = mne.events_from_annotations(eeg_loss_raw)
eeg_loss_events[:, 0] -= eeg_loss_raw.first_samp

In [ ]:
eeg_loss_event_dict

In [ ]:
#save eeg_loss_raw..
loss_path = project_path + pylossless_path + sync_loss_path
eeg_bids_path = qslt.write_eeg(eeg_loss_raw, 
              eeg_loss_event_dict, 
              eeg_loss_events, 
              subject_id, 
              session_id, 
              task_id, 
              loss_path, 
              device_info)
